In [1]:
import pvl, struct, quaternion
import numpy as np

In [2]:
"""
Read the binary data from an ISIS3 double table and return it as a dictionary

label : The ISIS3 Table label
file : The ISIS3 cube file
"""
def read_table(label, file):
    data_types = {
        'Integer' : {'format':'i', 'size':4},
        'Double'  : {'format':'d', 'size':8},
        'Real'    : {'format':'f', 'size':4},
        'Text'    : {'format':'c', 'size':1}
    }
    with open(file, 'rb') as f:
        f.seek(label['StartByte']-1) # Why is this -1 needed? It's taken straight from the ISIS source Blob::ReadData
        data = f.read(label['Bytes'])
    fields = label.getlist('Field')
    results = {field['Name']:[] for field in fields}
    offset = 0
    for record in range(label['Records']):
        for field in fields:
            count = field['Size']
            if field['Type'] == 'Text':
                results[field['Name']].append(data[offset:offset+count].decode(encoding='latin_1'))
            else:
                data_format = data_types[field['Type']]['format'] * count
                field_data = struct.unpack_from(data_format, data, offset)
                if len(field_data) == 1:
                    results[field['Name']].append(field_data[0])
                else:
                    results[field['Name']].append(field_data)
            offset += data_types[field['Type']]['size'] * count
    return results

In [28]:
def read_position_table(label, file):
    bin_data = read_table(label, file)
    results = {}
    if all (key in bin_data for key in ('J2000X','J2000Y','J2000Z')):
        results['Positions'] = np.array( [ [x, y, z] for x, y, z in zip(bin_data['J2000X'],bin_data['J2000Y'],bin_data['J2000Z']) ] )
    if 'ET' in bin_data:
        results['Times'] = np.array(bin_data['ET'])
    if all (key in bin_data for key in ('J2000XV','J2000YV','J2000ZV')):
        results['Velocities'] = np.array( [ [x, y, z] for x, y, z in zip(bin_data['J2000XV'],bin_data['J2000YV'],bin_data['J2000ZV']) ] )
    if all (key in bin_data for key in ('J2000SVX','J2000SVY','J2000SVZ')):
        results['PositionCoefficients'] = np.array( [bin_data['J2000SVX'][:-1],bin_data['J2000SVY'][:-1],bin_data['J2000SVZ'][:-1]] )
        results['BaseTime'] = bin_data['J2000SVX'][-1]
        results['TimeScale'] = bin_data['J2000SVY'][-1]
    return results

In [21]:
def read_rotation_table(label, file):
    bin_data = read_table(label, file)
    results = {}
    if all (key in bin_data for key in ('J2000Q0','J2000Q1','J2000Q2','J2000Q3')):
        results['Rotations'] = quaternion.as_quat_array( [ [q0, q1, q2, q2] for q0, q1, q2, q2 in zip(bin_data['J2000Q0'],bin_data['J2000Q1'],bin_data['J2000Q2'],bin_data['J2000Q3']) ] )
    if all (key in bin_data for key in ('AV1','AV2','AV3')):
        results['AngularVelocities'] = np.array( [ [av1, av2, av3] for av1, av2, av3 in zip(bin_data['AV1'],bin_data['AV2'],bin_data['AV3']) ] )
    if 'ET' in bin_data:
        results['Times'] = np.array(bin_data['ET'])
    if all (key in bin_data for key in ('J2000Ang1','J2000Ang2','J2000Ang3')):
        results['EulerCoefficients'] = np.array([bin_data['J2000Ang1'],bin_data['J2000Ang2'],bin_data['J2000Ang3']])
        results['BaseTime'] = bin_data['J2000Ang1'][-1]
        results['TimeScale'] = bin_data['J2000Ang2'][-1]
    if 'TimeDependentFrames' in label:
        results['TimeDependentFrames'] = np.array(label['TimeDependentFrames'])
    if all (key in label for key in ('ConstantRotation','ConstantFrames')):
        const_rotation_mat = np.array(label['ConstantRotation'])
        results['ConstantRotation'] = quaternion.from_rotation_matrix(np.reshape(const_rotation_mat, (3, 3)))
        results['ConstantFrames'] = np.array(label['ConstantFrames'])
    if all (key in label for key in ('PoleRa','PoleDec','PrimeMeridian')):
        results['BodyRotationCoefficients'] = np.array( [label['PoleRa'],label['PoleDec'],label['PrimeMeridian']] )
    if all (key in label for key in ('PoleRaNutPrec','PoleDecNutPrec','PmNutPrec','SysNutPrec0','SysNutPrec1')):
        results['SatelliteNutationPrecessionCoefficients'] = np.array( [label['PoleRaNutPrec'],label['PoleDecNutPrec'],label['PmNutPrec']] )
        results['PlanetNutationPrecessionAngleCoefficients'] = np.array( [label['SysNutPrec0'],label['SysNutPrec1']] )
    return results

In [5]:
cube_file = '/usgs/cpkgs/isis3/testData/isis/src/control/apps/jigsaw/tsts/radar/input/LSZ_00455_1CD_XKU_87S324_V1_S1_Null.cub'

In [6]:
label = pvl.load(cube_file)
tables = label.getlist('Table')
print(tables)

[PVLObject([
  ('Name', 'InstrumentPointing')
  ('StartByte', 136556582)
  ('Bytes', 76480)
  ('Records', 1195)
  ('ByteOrder', 'Lsb')
  ('TimeDependentFrames', [-85000, 1])
  ('ConstantFrames', [-85700, -85000])
  ('ConstantRotation',
   [1.0,
    0.0,
    0.0,
    0.0,
    0.67430238758372,
    0.73845534062588,
    0.0,
    -0.73845534062588,
    0.67430238758372])
  ('CkTableStartTime', 302388277.71971)
  ('CkTableEndTime', 302388516.48528)
  ('CkTableOriginalSize', 26302)
  ('Description', 'Created by spiceinit')
  ('Kernels',
   ['$lro/kernels/ck/moc42_2009213_2009214_v03.bc',
    '$lro/kernels/fk/lro_frames_2010277_v01.tf'])
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q0')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q1')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q2')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q3')
    ('Type', 'Double')
  

In [32]:
print(label)

PVLModule([
  ('IsisCube',
   {'Archive': {'DataSetId': 'LRO-L-MRFLRO-4-CDR-V1.0',
                'DataSetName': 'LRO MOON MINI-RF 4 CALIBRATED DATA RECORD V1.0',
                'ProducerFullName': 'MINI-RF POC TEAM',
                'ProducerId': 'JHUAPL',
                'ProducerInstitutionName': 'JOHNS HOPKINS UNIVERSITY APPLIED '
                                           'PHYSICS LABORATORY',
                'ProductCreationTime': datetime.datetime(2010, 2, 6, 12, 21, 33),
                'ProductId': 'LSZ_00455_1CD_XKU_87S324_V1',
                'ProductVersionId': 1.0,
                'ReleaseId': 1,
                'SoftwareName': 'FOCUS',
                'SoftwareVersionId': '9.0.320'},
    'BandBin': PVLGroup([
    ('FilterName', 'H RECEIVE INTENSITY')
  ]),
    'Core': {'Dimensions': {'Bands': 1,
                            'Lines': 26301,
                            'Samples': 1241},
             'Format': 'Tile',
             'Pixels': {'Base': 0.0,
                   

In [7]:
read_position_table(tables[1], cube_file)

{'Positions': [[-156.3249467565359, 42.73228116264915, 241.27240232500955],
  [659.679558446114, 692.9027317007598, 717.5515171234637],
  [-1645.1440297221175, -1638.4398876547368, -1611.4422933973376]],
 'Times': [302388277.7197137, 302388397.097959, 302388516.4852824],
 'Velocities': [[1.6627570662146614, 1.6686724581040384, 1.6538810102678987],
  [0.3131519168739245, 0.2428710020222609, 0.16965810979736903],
  [-0.02926015404243728, 0.14146863974779417, 0.31028387905784444]]}

In [22]:
read_rotation_table(tables[0], cube_file)

{'Rotations': array([quaternion(0.0896823016071204, 0.0273732804905333, 0.976851297505324, 0.976851297505324),
        quaternion(0.0896641027716965, 0.0272801808432849, 0.976853820196503, 0.976853820196503),
        quaternion(0.0896459034348931, 0.0271879510417398, 0.976856447429457, 0.976856447429457),
        ...,
        quaternion(0.0673974024425705, -0.0832839030183182, 0.973676235287308, 0.973676235287308),
        quaternion(0.0673784994848749, -0.083376799362564, 0.973668272556068, 0.973668272556068),
        quaternion(0.0673595011925386, -0.0834692014777462, 0.97366036723777, 0.97366036723777)],
       dtype=quaternion),
 'AngularVelocities': array([[ 0.00016062, -0.00086711, -0.00036146],
        [ 0.0001528 , -0.00085933, -0.00035972],
        [ 0.00015679, -0.00087885, -0.00036875],
        ...,
        [ 0.0001566 , -0.00086882, -0.00036026],
        [ 0.0001554 , -0.00086382, -0.00036031],
        [ 0.00015729, -0.00086722, -0.00036805]]),
 'Times': array([3.02388278e+

In [23]:
read_rotation_table(tables[2], cube_file)

{'Rotations': array([quaternion(0.782644033366781, -0.16405498562924, -0.590560244701748, -0.590560244701748),
        quaternion(0.782456358301271, -0.16402032872713, -0.590808912387147, -0.590808912387147)],
       dtype=quaternion),
 'AngularVelocities': array([[ 6.25422931e-08, -1.02584300e-06,  2.45538340e-06],
        [ 6.25425062e-08, -1.02584307e-06,  2.45538345e-06]]),
 'Times': array([3.02388278e+08, 3.02388516e+08]),
 'TimeDependentFrames': array([31006,     1]),
 'ConstantRotation': quaternion(0.99999996831339, 7.58574205202439e-07, 0.000190434690476096, 0.000164642860863603),
 'ConstantFrames': array([31001, 31007, 31006])}

In [31]:
read_position_table(tables[3], cube_file)

{'Positions': array([[-9.56856061e+07,  1.08444791e+08,  4.70408309e+07],
        [-9.56912322e+07,  1.08440755e+08,  4.70390643e+07]]),
 'Times': array([3.02388278e+08, 3.02388516e+08]),
 'Velocities': array([[-23.56367798, -16.90426244,  -7.39840375],
        [-23.56293523, -16.9057504 ,  -7.39908848]])}